In [ ]:
# load dependencies
from matplotlib import pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
import scipy.stats as st
import seaborn as sns

In [ ]:
# read in files
happiness_2018_index = pd.read_csv('../Data/happiness_index_2018.csv')
alcohol_consumption = pd.read_csv('../Data/alcohol_consumption.csv')
consumption_type = pd.read_csv('../Data/drinks_csv.csv')
gdp = pd.read_csv('../Data/gdp.csv')

# merge csv files
df = pd.merge(happiness_2018_index, alcohol_consumption, on='Country', how='outer')
df = pd.merge(df, consumption_type, on='Country', how='outer')
df = pd.merge(df, gdp, on='Country', how='outer')


# check the file has been read in
df

In [ ]:
# clean dataframe with only the relevant data
df = df[['Country', 'Country Code_x', 'Score', '2018_y',"Social support", 'Healthy life expectancy', "Freedom to make life choices",
         'Generosity', 'Perceptions of corruption','2018_x', 'beer_servings', 'spirit_servings', 'wine_servings']]

# check dataframe for any errors
df = df.dropna()

# rename columns

df =df.rename(columns={"Country Code_x": "con_code", 
                       "2018_y": "GDP", 
                       "Social support": "social",
                       "Healthy life expectancy": "life_expectancy",
                       "Freedom to make life choices": "freedom",
                       'Perceptions of corruption': "corruption",
                       "2018_x": "alcohol_consumption"
                      })

df

In [ ]:
# data check to see if this is an interesting enough correlation to explore
alcohol_c = df["alcohol_consumption"]
score = df['Score']

# create and format plot
plt.scatter(alcohol_c, score, marker="o", facecolors="red", edgecolors="black")

# show plot
plt.show()
plt.tight_layout()


In [ ]:
# calculating correlation score

corr = st.pearsonr(alcohol_c,score)

corr

In [ ]:
df.describe()

In [ ]:
happyCountries = df[(df.Score > 7)]

happyCountries.describe()

In [ ]:
sadCountries = df[(df.Score < 4)]

sadCountries.describe()

### CREATE DATABASE CONNECTION

In [ ]:
#create connection
import psycopg2

Load Data into Postgres

In [ ]:
# create params_dic
param_dic = {
    "host"      : "non-alcoholics.cexcs0a519gc.us-west-1.rds.amazonaws.com",
    "database"  : "non-alcoholics",
    "user"      : "postgres",
    "password"  : "postgres"
}

In [ ]:
# set up connection
connect = "postgresql+psycopg2://%s:%s@%s:5432/%s"%(
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)

engine = create_engine(connect)

In [ ]:
conn = engine.connect()

results = conn.execute('Select * from df')


In [ ]:
all_countries = []
 
for country, con_code, score, gdp, social, health, freedom, generosity, corruption, alcohol, beer_servings, wine_servings, spirit_servings in results:
    print(country)
    countries_dict = {}
    countries_dict["Country"] = country
    countries_dict["Country Code"] = con_code
    countries_dict["Score"] = score
    countries_dict["GDP per capita"] = gdp
    countries_dict["Social support"] = social
    countries_dict["Health life expectancy"] = health
    countries_dict["Freedom to make life choices"] = freedom
    countries_dict["Generosity"] = generosity
    countries_dict["Perceptions of corruption"] = corruption
    countries_dict["Alcohol Consumption per Capita (liter)"] = alcohol
    countries_dict["beer_servings"] = beer_servings
    countries_dict["wine_servings"] = wine_servings
    countries_dict["spirit_servings"] = beer_servings
    all_countries.append(countries_dict)



print(all_countries)

In [ ]:
def to_alchemy(df):
    """
    Reading table into postgres
    """
df.to_sql(
        'df', 
        con=engine, 
        index=False, 
        if_exists='replace'
    )
print("to_sql() done (sqlalchemy)")

In [ ]:
df.head()